# Notebook to generate target quarter for 2025

In [2]:
import pandas as pd
import plotly.express as px
import json

## Load data

In [3]:
df = pd.read_csv('payments_and_pledges.csv')
df['date'] = pd.to_datetime(df['date'])

In [4]:
df.columns

Index(['id', 'donor_id', 'payment_platform', 'portfolio', 'amount', 'currency',
       'date', 'counterfactuality', 'pledge_id', 'multiplier', 'amount_usd',
       'year', 'month', 'donor_chapter', 'chapter_type', 'pledge_status',
       'pledge_created_at', 'pledge_starts_at', 'pledge_ended_at',
       'contribution_amount', 'frequency'],
      dtype='object')

## Transform data

### Add quarter

- `Civil quarter`

In [5]:
df['quarter_civil'] = df['date'].dt.quarter

- `Fiscal quarter` and `fiscal year`

In [6]:
def compute_fiscal_quarter(date):
    month = date.month
    return ((month - 7) % 12) // 3 + 1

def compute_fiscal_year(date):
    return date.year + 1 if date.month >= 7 else date.year

df["fiscal_quarter"] = df["date"].apply(compute_fiscal_quarter)
df["fiscal_year"] = df["date"].apply(compute_fiscal_year)

In [7]:
df.head()

,id,donor_id,payment_platform,portfolio,amount,currency,date,counterfactuality,pledge_id,multiplier,...,chapter_type,pledge_status,pledge_created_at,pledge_starts_at,pledge_ended_at,contribution_amount,frequency,quarter_civil,fiscal_quarter,fiscal_year
0,5B5XGD4RTX,NaN,Benevity,OFTW Top Picks,-100.00,USD,2024-02-01,0.313636,7373fca9-78cc-4954-baaa-b2acebac595b,1.0,...,NaN,One-Time,2024-02-01,2024-02-01,NaN,-100.00,One-Time,1,3,2024
1,3Z5NNTTCSD,NaN,Benevity,OFTW Top Picks,-1.01,USD,2022-09-23,0.313636,5e6bc702-7990-4351-a5b5-4cf39401c379,1.0,...,Corporate,One-Time,2022-09-23,2022-09-23,NaN,-1.01,One-Time,3,1,2023
2,31V9VD2AAV,NaN,Benevity,OFTW Top Picks,0.00,USD,2021-11-19,0.313636,8a8c1bc3-a6a4-4819-a8c1-732e7a88408b,1.0,...,NaN,One-Time,2021-11-19,2021-11-19,NaN,0.00,One-Time,4,2,2022
3,31V9VJBAN5,NaN,Benevity,OFTW Top Picks,0.00,USD,2021-11-19,0.313636,8a8c1bc3-a6a4-4819-a8c1-732e7a88408b,1.0,...,NaN,One-Time,2021-11-19,2021-11-19,NaN,0.00,One-Time,4,2,2022
4,31V9VMC0YD,NaN,Benevity,OFTW Top Picks,0.00,USD,2021-11-19,0.313636,d16156a3-e981-4019-80ae-bceb2daaf047,1.0,...,NaN,One-Time,2021-11-19,2021-11-19,NaN,0.00,One-Time,4,2,2022


### Functions to include revelant years

In [8]:
def get_complete_years(df: pd.DataFrame, year_col: str, quarter_col: str) -> list:
    """
    Returns years where all 4 quarters are present.

    Parameters:
        df: DataFrame with 'year' or 'fiscal_year' and 'quarter' columns.
        year_col: 'year' or 'fiscal_year'
        quarter_col: 'quarter_civil' or 'fiscal_quarter'

    Returns:
        List of years with all 4 quarters present.
    """
    df_grouped = df.groupby(year_col)[quarter_col].nunique().reset_index(name='total_nb_quarters')
    return sorted(df_grouped.query("total_nb_quarters == 4")[year_col].to_list())

In [9]:
def get_years_with_enough_data(df: pd.DataFrame, year_col: str, min_rows: int = 1000) -> list:
    """
    Returns a list of years that have at least `min_rows` entries.

    Args:
        df (pd.DataFrame): The input DataFrame.
        year_col (str): The name of the column containing year information.
        min_rows (int): Minimum number of rows required for a year to be considered valid.

    Returns:
        list: Sorted list of years with enough data.
    """
    return sorted(df[year_col].value_counts().loc[lambda x: x >= min_rows].index.tolist())

#### Why I picked `min_rows=1000`

I chose **1000** as a threshold because, from exploring the dataset, the most "complete" and reliable years have more than 1000 entries. Anything below that usually signals partial or less trustworthy data. It’s a pragmatic choice for now, but of course, this value can easily be adjusted later based on more robust statistical analysis or data-specific heuristics.


In [10]:
def filter_valid_years(df: pd.DataFrame, year_col: str, quarter_col: str, min_rows: int = 1000) -> pd.DataFrame:
    """
    Filters the DataFrame to include only years that are:
    - Complete (i.e., have all 4 quarters)
    - Have at least `min_rows` entries

    Args:
        df (pd.DataFrame): The input DataFrame.
        year_col (str): The column representing the year (fiscal or calendar).
        quarter_col (str): The column representing the quarter (fiscal or calendar).
        min_rows (int): Minimum number of entries required per year.

    Returns:
        pd.DataFrame: A filtered DataFrame with only valid years.
    """
    complete_years = get_complete_years(df, year_col=year_col, quarter_col=quarter_col)
    years_with_enough_data = get_years_with_enough_data(df, year_col=year_col, min_rows=min_rows)
    valid_years = list(set(complete_years).intersection(years_with_enough_data))
    return df.query(f'{year_col} in @valid_years')

## Compute proportions for each metrics

In [11]:
targets_2025 = {
    "money_moved": 1_800_000,
    "counterfactual_mm": 1_260_000,
    "active_arr": 1_200_000,
    "pledge_attrition_rate": 18,
    "total_active_donors": 1200,
    "total_active_pledges": 850,
    "all_pledges": 1850,
    "future_pledges": 1000,
    "active_pledges": 850,
    "all_arr": 1_800_000,
    "future_arr": 600_000,
    "monthly_attrition": 18
}

In [12]:
metrics_to_analyze = {
    "money_moved": {"column": "amount_usd", "method": "sum"},
    "counterfactual_mm": {"column": ("amount_usd", "counterfactuality"), "method": "counterfactual_sum"},
    "active_arr": {"column": "amount_usd", "method": "sum", "filter": ["Active donor"]},
    "pledge_attrition_rate": {"column": "pledge_status", "method": "rate", "filter": ["Payment failure", "Churned donor"], "exclude_status": ["ERROR"]},
    "total_active_donors": {"column": "donor_id", "method": "nunique", "filter": ['Active donor', 'One-Time']},
    "total_active_pledges": {"column": "donor_id", "method": "nunique", "filter": ["Active donor"]},
    "all_pledges": {"column": "donor_id", "method": "nunique", "filter": ["Active donor", "Pledged donor"]},
    "future_pledges": {"column": "donor_id", "method": "nunique", "filter": ["Pledged donor"]},
    "active_pledges": {"column": "donor_id", "method": "nunique", "filter": ["Active donor"]},
    "all_arr": {"column": "amount_usd", "method": "sum", "filter": ["Active donor", "Pledged donor"]},
    "future_arr": {"column": "amount_usd", "method": "sum", "filter": ["Pledged donor"]},
    "monthly_attrition": {"column": "pledge_status", "method": "rate", "filter": ["Payment failure", "Churned donor"], "exclude_status": ["ERROR"]}
}

In [13]:
def compute_additive_metric_proportion(
          df_filtered, 
          metric_fn_agg: str, 
          metric_column: str,
          year_type: str,
          quarter_type: str
) -> dict:
        """
        Computes average quarterly distribution proportions for additive or countable metrics.

        This function is designed specifically for absolute metrics (e.g., sums, unique counts, 
        counterfactual sums), and calculates what percentage each quarter historically contributes 
        to the annual total. These proportions are useful to estimate quarterly targets based 
        on an annual goal.

        This function is NOT suitable for percentage-based metrics like rates or ratios. 
        Use `compute_rate_metric` instead.

        Args:
            df_filtered (pd.DataFrame): Filtered dataset containing the metric data.
            metric_fn_agg (str): Aggregation method ('sum', 'nunique', or 'counterfactual_sum').
            metric_column (str): Name of the column being aggregated.
            year_type (str): The year grouping column ('year' or 'fiscal_year').
            quarter_type (str): The quarter grouping column ('quarter_civil' or 'fiscal_quarter').

        Returns:
            dict: Mapping of quarter numbers (1–4) to their average percentage contribution 
                to the total annual metric, rounded to 1 decimal place.
        """
        agg_col_name = f"{metric_column}_{metric_fn_agg}"

        if metric_fn_agg == 'counterfactual_sum':
            # Group with custom lambda for counterfactual logic
            df_grouped = (
                df_filtered
                .groupby([year_type, quarter_type])
                .apply(lambda x: (x['amount_usd'] * x['counterfactuality']).sum(), include_groups=False)
                .reset_index(name=agg_col_name)
            )
        else: 
            # Regular aggregation for sum/nunique
            df_grouped = (
                df_filtered
                .groupby([year_type, quarter_type])[metric_column]
                .agg(metric_fn_agg)
                .reset_index(name=agg_col_name)
            )

        # 2. Calculate annual total (dynamically based on same col)
        df_annual = df_grouped.groupby(year_type)[agg_col_name].sum().reset_index(name='total_annual')

        # 3. Merge to get proportions
        df_merged = df_grouped.merge(df_annual, how='inner', on=year_type)
        df_merged['proportion'] = round((df_merged[agg_col_name] / df_merged['total_annual']) * 100, 1)

        # 4. Get average proportion by quarter
        quarter_metric_proportion_avg = (
            df_merged.groupby(quarter_type)['proportion'].mean().round(1).to_dict()
        )

        return quarter_metric_proportion_avg

In [14]:
def compute_rate_metric_proportion(
    df_filtered: pd.DataFrame,
    metric_fn_agg: str, 
    metric_column: str,
    year_type: str,
    quarter_type: str,
) -> dict:
    """
    Computes the historical average share of each quarter in the total annual value
    for rate-based metrics (e.g., attrition rate).

    This function calculates what portion of a rate-related metric (like number of failed pledges)
    typically occurs in each quarter, helping estimate realistic quarterly targets 
    based on an annual rate goal.

    Only use this function for metrics based on relative frequencies, not absolute totals 
    like monetary values or counts — use `compute_additive_metric_proportion` for those.

    Args:
        df_filtered (pd.DataFrame): Pre-filtered DataFrame (e.g., filtered by status).
        metric_fn_agg (str): Aggregation name for context ('rate' or similar).
        metric_column (str): Metric-related column (typically a status column).
        year_type (str): Year column to group by ('year' or 'fiscal_year').
        quarter_type (str): Quarter column to group by ('quarter_civil' or 'fiscal_quarter').

    Returns:
        dict: Dictionary mapping each quarter (1–4) to its average contribution (in %) 
              to the total annual metric, rounded to 1 decimal.
    """

    agg_col_name = f"{metric_column}_{metric_fn_agg}"

    # Step 1: Count quarterly occurrences (e.g., nb of attrition cases)
    df_quarterly = (
        df_filtered
        .groupby([year_type, quarter_type])
        .size()
        .reset_index(name=agg_col_name)
    )

    # Step 2: Compute annual totals
    df_annual_total = (
        df_quarterly
        .groupby(year_type)[agg_col_name]
        .sum()
        .reset_index(name='total_annual')
    )

    # Step 3: Merge and compute proportions
    df_with_total = df_quarterly.merge(df_annual_total, on=year_type, how='inner')
    df_with_total['proportion'] = round((df_with_total[agg_col_name] / df_with_total['total_annual']) * 100, 1)

    # Step 4: Average proportion per quarter across all years
    quarter_avg_proportion = (
        df_with_total
        .groupby(quarter_type)['proportion']
        .mean()
        .round(1)
        .to_dict()
    )

    return quarter_avg_proportion


In [15]:
KEY_QUARTER_PROPORTION_CIVIL = 'quarter_proportion_civil'
KEY_QUARTER_PROPORTION_FISCAL = 'quarter_proportion_fiscal'

In [16]:
df.columns

Index(['id', 'donor_id', 'payment_platform', 'portfolio', 'amount', 'currency',
       'date', 'counterfactuality', 'pledge_id', 'multiplier', 'amount_usd',
       'year', 'month', 'donor_chapter', 'chapter_type', 'pledge_status',
       'pledge_created_at', 'pledge_starts_at', 'pledge_ended_at',
       'contribution_amount', 'frequency', 'quarter_civil', 'fiscal_quarter',
       'fiscal_year'],
      dtype='object')

In [17]:
df_civil = filter_valid_years(df=df, year_col='year', quarter_col='quarter_civil', min_rows=1000)
df_fiscal = filter_valid_years(df=df, year_col='fiscal_year', quarter_col='fiscal_quarter', min_rows=1000)

In [23]:
all_metrics_target = {}

for metric_name, params in metrics_to_analyze.items():

    print('------------------------------------------------')
    print('Metric name:', metric_name)
    print('Params:', params)

    metric_fn_agg = params['method']
    metric_column = params['column']
    status_to_filter = params.get('filter', '')
    status_to_exclude = params.get('exclude_status')
    all_metrics_target[metric_name] = {'target_annual': targets_2025.get(metric_name)}
    
    df_filtered = df.copy()

    if status_to_filter:
        df_filtered = df.query("pledge_status in @status_to_filter")
    if status_to_exclude:
        df_filtered = df.query("pledge_status not in @status_to_exclude")


    # Standard metric
    if metric_fn_agg in ['sum', 'counterfactual_sum', 'nunique']:

        # Civil Year
        all_metrics_target[metric_name][KEY_QUARTER_PROPORTION_CIVIL] = compute_additive_metric_proportion(
            df_filtered=df_civil,
            metric_column=metric_column,
            metric_fn_agg=metric_fn_agg,
            year_type='year',
            quarter_type='quarter_civil'
        )

        # Fiscal Year
        all_metrics_target[metric_name][KEY_QUARTER_PROPORTION_FISCAL] = compute_additive_metric_proportion(
            df_filtered=df_fiscal,
            metric_column=metric_column,
            metric_fn_agg=metric_fn_agg,
            year_type='fiscal_year',
            quarter_type='fiscal_quarter'
        )

    # Rate metric
    else:

        # Civil Year
        all_metrics_target[metric_name][KEY_QUARTER_PROPORTION_CIVIL] = compute_rate_metric_proportion(
            df_filtered=df_civil,
            year_type='year',
            quarter_type='quarter_civil',
            metric_fn_agg=metric_fn_agg,
            metric_column=metric_column
        )

        # Fiscal Year
        all_metrics_target[metric_name][KEY_QUARTER_PROPORTION_FISCAL] = compute_rate_metric_proportion(
            df_filtered=df_fiscal,
            year_type='fiscal_year',
            quarter_type='fiscal_quarter',
            metric_fn_agg=metric_fn_agg,
            metric_column=metric_column
        )


------------------------------------------------
Metric name: money_moved
Params: {'column': 'amount_usd', 'method': 'sum'}
------------------------------------------------
Metric name: counterfactual_mm
Params: {'column': ('amount_usd', 'counterfactuality'), 'method': 'counterfactual_sum'}
------------------------------------------------
Metric name: active_arr
Params: {'column': 'amount_usd', 'method': 'sum', 'filter': ['Active donor']}
------------------------------------------------
Metric name: pledge_attrition_rate
Params: {'column': 'pledge_status', 'method': 'rate', 'filter': ['Payment failure', 'Churned donor'], 'exclude_status': ['ERROR']}
------------------------------------------------
Metric name: total_active_donors
Params: {'column': 'donor_id', 'method': 'nunique', 'filter': ['Active donor', 'One-Time']}
------------------------------------------------
Metric name: total_active_pledges
Params: {'column': 'donor_id', 'method': 'nunique', 'filter': ['Active donor']}
-----

In [20]:
import pprint
pp = pprint.PrettyPrinter()

In [22]:
YEAR_TARGET = 2025

In [24]:
final_targets_json = {YEAR_TARGET: all_metrics_target}

In [25]:
pp.pprint(final_targets_json)

{2025: {'active_arr': {'quarter_proportion_civil': {1: 19.9,
                                                    2: 18.3,
                                                    3: 19.6,
                                                    4: 42.1},
                       'quarter_proportion_fiscal': {1: 16.3,
                                                     2: 37.4,
                                                     3: 24.2,
                                                     4: 22.1},
                       'target_annual': 1200000},
        'active_pledges': {'quarter_proportion_civil': {1: 21.4,
                                                        2: 24.6,
                                                        3: 25.2,
                                                        4: 28.8},
                           'quarter_proportion_fiscal': {1: 21.6,
                                                         2: 25.5,
                                                         3: 24.

In [26]:
with open('targets.json', 'w') as f:
    json.dump(final_targets_json, f, indent=4)